In [1]:
import pandas as pd
import numpy as np
import pickle
from collections import defaultdict
from collections import Counter
import math
import tqdm
import ast

In [2]:
#saving dict function
def save_dict(obj, name ):
    with open(name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

#load dict function
def load_obj(name ):
    with open(name + '.pkl', 'rb') as f:
        return pickle.load(f)

## Naïve Bayes Multinomial Event Model

In [3]:
train = pd.read_csv('mapped_train.csv')
train.drop({'Unnamed: 0'},axis=1,inplace=True)

In [4]:
train.head()

,id,text,label,mapped_text
0,5204,sarah palin medal honor recipient son law anni...,1,"[24136, 8678, 15272, 86830, 66089, 13273, 3297..."
1,9143,wow un ambassador nikki haley blast un vote tw...,1,"[62682, 25830, 84783, 62204, 76717, 87416, 258..."
2,35861,brazil police raid home two lawmaker bribery p...,0,"[88061, 54921, 31378, 2842, 50501, 80489, 3884..."
3,35051,factbox next catalonia exile separatist leader...,0,"[64859, 85537, 56046, 666, 69815, 38455, 49260..."
4,3433,trump run scar cancel press conference russia ...,1,"[81454, 15501, 87072, 77293, 40035, 28956, 302..."


In [5]:
train_0 = train.loc[train['label']==0]
train_1 = train.loc[train['label']==1]

In [6]:
word_index = load_obj('word_index')

In [7]:
len(word_index)

89369

In [8]:
phi_y = train['label'].sum()/len(train)
phi_y

0.5221337490951612

In [9]:
#0 real 1 fake
n = len(word_index)

phi_0 = np.ones(n) 
phi_1 = np.ones(n)

d_0 = sum([len(i) for i in train_0.mapped_text])
d_1 = sum([len(i) for i in train_1.mapped_text])

for word, value in word_index.items():
    for tup in value:
        if tup[2] == 0:
            phi_0[word] += tup[1]
        else:
            phi_1[word] += tup[1]
phi_0 = phi_0/(d_0+n)
phi_1 = phi_1/(d_1+n)

In [10]:
phi_1[0:100]

array([2.15976691e-07, 1.54269065e-07, 6.17076259e-08, 6.17076259e-08,
       8.94760575e-07, 9.25614388e-08, 3.08538129e-08, 6.17076259e-08,
       9.25614388e-08, 3.08538129e-08, 6.17076259e-08, 6.17076259e-08,
       3.08538129e-08, 1.51183683e-06, 6.17076259e-08, 1.85122878e-07,
       3.08538129e-08, 6.17076259e-08, 2.15976691e-07, 9.25614388e-08,
       6.17076259e-08, 9.25614388e-08, 2.77684316e-07, 4.62807194e-07,
       3.08538129e-08, 9.25614388e-07, 1.70004509e-05, 9.25614388e-08,
       3.21188193e-05, 6.17076259e-08, 6.17076259e-08, 6.17076259e-08,
       3.39391942e-07, 6.17076259e-08, 9.25614388e-08, 6.17076259e-08,
       3.08538129e-08, 9.25614388e-08, 2.40659741e-06, 6.47930072e-07,
       1.54269065e-07, 6.17076259e-08, 3.08538129e-08, 9.25614388e-08,
       6.17076259e-08, 3.08538129e-08, 3.08538129e-08, 1.24032328e-05,
       3.08538129e-08, 2.15976691e-07, 1.23415252e-07, 3.08538129e-08,
       1.35139701e-05, 9.25614388e-08, 9.25614388e-08, 9.25614388e-08,
      